In [2]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
import codecs
import time
import re
import os
from IPython.display import display, HTML

In [27]:
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"

In [28]:
# headers = {
#     'User-Agent': 'Your Name, example.com',
#     'From': 'email@example.com'
# }

In [22]:

os.makedirs('workdata/lines_submit/wronginput/output/', exist_ok=True)
os.makedirs('workdata/lines_submit/wrong_lines/output/', exist_ok=True)
os.makedirs('workdata/lines_submit/correct_lines/output/', exist_ok=True)

pbwords = []
fnames = ["all_uniquelines.txt"]
count = 0     
fw1 = open("workdata/lines_submit/wronginput/output/wronginput.txt",encoding = 'utf-8' ,mode ='a')
for fname in fnames :
    problem = []
    problem.append(-1)
    fr = open("workdata/lines_submit/"+fname,encoding ='utf-8',mode='r')
    _lines = fr.readlines()
    fr.close()
    
    for _line in _lines :
        
        problem = []
        problem.append(-1)
        if(_line.startswith("filename : ")) :
            line1 = _line
            continue
        if(_line.startswith("lineno : ")) :
            line2 = _line
            continue
        if(_line.startswith("line : ")) :
            line3 = _line[7:]
            count += 1
#             if(count<3558) :
                
#                 continue
            time.sleep(2)
            _line1 = line3[:-1]
            _line2 = _line1 # re.sub(r'\d+|\"|\?|\.|\,|\*|\!|_\?|\-|_\ |\'',"",_line1)
            inputline = _line2
            if(inputline == " ") :
                print("emptyline")
                count = count-1
                continue
            s_d = inputline
            print(count)
            print(_line1)
            print(inputline)
#             fw = codecs.open("workdata/linetabledata/"+ str(count)+".txt",encoding = 'utf-8',mode = 'w')
#             fw.writelines("given line : " +_line1 + "\n")                                                        #file input
            s_c = s_d.replace(" ","+")
            urlname = ("http://sanskrit.inria.fr/cgi-bin/SKT/sktgraph?lex=SH&st=t&us=f&cp=t&text=" +
                       s_c + "&t=WX&topic=&mode=g&corpmode=&corpdir=&sentno=")

            print(urlname)
            page = requests.get(urlname)
            soup = bs(page.text, 'html.parser')
        #     soup.prettify()
            table = soup.table
            tablebody = table.find('table',{'class' : 'center'})
            t = pd.DataFrame(columns = ['id','level','color_class','position','chunk_no','word','lemma','pre_verb','morph','colspan','wordlenth','aux_inf'])

            i = 0
            id_= 0
            if not(tablebody) :      #### wronginputs
                fw1.writelines(line1)
                fw1.writelines(line2)
                fw1.writelines(line3)
                fw1.writelines("count =" +str(count)+"\n")
                continue
            for child in tablebody.children:
                if(child.name == 'tr') :
                    if i< 1 : 
                        linechar = []
                        c=0
                        for char in child.children :
                            linechar.append(char.string)
                            c += 1
                        i+=1
                        line = "".join(linechar)
                        linechunks = line.split("\xa0")
                        continue
                    position_ = 0
                    j =0
                    for wordtable in child.children:
            #             if(j < 20 ) :
            #                 j += 1
            #                 continue
                        c= 0
                        for ch in linechar[0:position_] :
                            if(re.match('\xa0',ch)) : 
                                c+=1
                        if(wordtable.contents):
                            color_ = wordtable.table.get('class')[0]
                            colspan_= wordtable.get('colspan')
                            word_ = wordtable.table.tr.td.string
                            onclickdatas_ = wordtable.table.tr.td.get('onclick')
                            for onclickdata_ in onclickdatas_.split("<br>") :
                                morphslist_ = re.findall(r'{ (.*?) }',onclickdata_)  #.split(' | ')
                                ldata = str(re.search(r'{.*?}\[(.*)\]',onclickdata_).group(1))
                                ldata = str(re.sub(r'</?a.*?>|</?i>',"",ldata))

                                lemmadata = ldata.split(" ")
                                if len(lemmadata) >1 :
                                    auxi_ = " ".join(lemmadata[1:])
                                else :
                                    auxi_ = ""
                                lemmas_ = "".join(lemmadata[0])
                                lemmalists_ = lemmas_.split("-")
                                if(len(lemmalists_) > 1) :
                                    preverb_ = ",".join(lemmalists_[0:(len(lemmalists_)-1)])
                                    lemmalist_ = "".join(lemmalists_[-1:]).split("_")
                                else :
                                    preverb_ = ""
                                    lemmalist_ = "".join(lemmalists_[0]).split("_")
                                if(len(lemmalist_) > 1):
                                    auxi_ =auxi_ + " sence of lemma = " + "".join(lemmalist_[1:(len(lemmalist_))])
                                    lemma_ = "".join(lemmalist_[0])
                                else : 
                                    lemma_ = "".join(lemmalist_[0])             
                                morphs_ = str(morphslist_[0])               
                                for morph_ in morphs_.split(" | ") : 
                                    t.loc[id_] = [id_,i,color_,position_,c+1,word_,lemma_,preverb_,morph_,colspan_,len(word_),auxi_]
                                    if(re.match(r'grey_back',color_)) :
                                        problem.append(id_)
                                        if word_ not in pbwords :
                                            pbwords.append(word_)
                                    id_ += 1    


                            position_ += int(colspan_)
                #                 break
                        else : 
                            position_ += 1

            #         break

                    i = i+1
            if(len(problem)>1) :
                fn = "workdata/lines_submit/wrong_lines/output/"+ str(count)+".txt"
            else :
                fn = "workdata/lines_submit/correct_lines/output/"+ str(count)+".txt"
            
            fw = open(fn,encoding = 'utf-8' ,mode ='w')
            fw.writelines(line1)
            fw.writelines(line2)
#             fw.writelines("given line : " +_line1 + "\n")
            fw.writelines("input line : " + inputline + "\n")
            fw.writelines("converted line : "+line + "\n")
            fw.writelines("converted line char :" + ",".join(linechar) + "\n")
            if(len(problem)>1) :
                _prob = []
                for prob in problem[1:]:
                    _prob.append(str(prob))
                fw.writelines("problem_yes ,index = ["+"-".join(_prob) + "]\n")
            else :
                fw.writelines("problem_no" + "\n")
            fw.close()
            t.to_csv(fn ,encoding = 'utf-8' , sep=',',index = False,mode = 'a' )
    #         display(t)
    #          count += 1 
    #         if(count > 2) :
    #             break
    #     break
fw1.close()
# fr = open("data/sansk.txt",'r')
# print(fr.readlines())
# fr.close()

3558
kAruNikaH SamIkaH sawwvavanwaH AyuRmanwaH ca ime sanwi iwi ukwvA wAn svASrame AnIya yaWociwAhArapAnapraxAnAxinA apoRayaw 
kAruNikaH SamIkaH sawwvavanwaH AyuRmanwaH ca ime sanwi iwi ukwvA wAn svASrame AnIya yaWociwAhArapAnapraxAnAxinA apoRayaw 
http://sanskrit.inria.fr/cgi-bin/SKT/sktgraph?lex=SH&st=t&us=f&cp=t&text=kAruNikaH+SamIkaH+sawwvavanwaH+AyuRmanwaH+ca+ime+sanwi+iwi+ukwvA+wAn+svASrame+AnIya+yaWociwAhArapAnapraxAnAxinA+apoRayaw+&t=WX&topic=&mode=g&corpmode=&corpdir=&sentno=
3559
yaWAsamayaM jAwapakRAH we akasmAw ekasmin xine uxadIyanwa 
yaWAsamayaM jAwapakRAH we akasmAw ekasmin xine uxadIyanwa 
http://sanskrit.inria.fr/cgi-bin/SKT/sktgraph?lex=SH&st=t&us=f&cp=t&text=yaWAsamayaM+jAwapakRAH+we+akasmAw+ekasmin+xine+uxadIyanwa+&t=WX&topic=&mode=g&corpmode=&corpdir=&sentno=
3560
we eva keRuciw mAseRu parAvqwya AwmAnaM paricAyayanwaH 
we eva keRuciw mAseRu parAvqwya AwmAnaM paricAyayanwaH 
http://sanskrit.inria.fr/cgi-bin/SKT/sktgraph?lex=SH&st=t&us=f&cp=t&text=we+eva+keRuciw+mAse

3584
anena KinnaH BAraviH piwaraM hanwuM nilInaH 
anena KinnaH BAraviH piwaraM hanwuM nilInaH 
http://sanskrit.inria.fr/cgi-bin/SKT/sktgraph?lex=SH&st=t&us=f&cp=t&text=anena+KinnaH+BAraviH+piwaraM+hanwuM+nilInaH+&t=WX&topic=&mode=g&corpmode=&corpdir=&sentno=
3585
rAwrO wasya mAwrA pqRtaH 
rAwrO wasya mAwrA pqRtaH 
http://sanskrit.inria.fr/cgi-bin/SKT/sktgraph?lex=SH&st=t&us=f&cp=t&text=rAwrO+wasya+mAwrA+pqRtaH+&t=WX&topic=&mode=g&corpmode=&corpdir=&sentno=
3586
piwA kaWiwavAn 
piwA kaWiwavAn 
http://sanskrit.inria.fr/cgi-bin/SKT/sktgraph?lex=SH&st=t&us=f&cp=t&text=piwA+kaWiwavAn+&t=WX&topic=&mode=g&corpmode=&corpdir=&sentno=
3587
BAraviH viSvaviKyAwaH ekaH ajaraH amaraH mahAkaviH Bavawu 
BAraviH viSvaviKyAwaH ekaH ajaraH amaraH mahAkaviH Bavawu 
http://sanskrit.inria.fr/cgi-bin/SKT/sktgraph?lex=SH&st=t&us=f&cp=t&text=BAraviH+viSvaviKyAwaH+ekaH+ajaraH+amaraH+mahAkaviH+Bavawu+&t=WX&topic=&mode=g&corpmode=&corpdir=&sentno=
3588
ixaM vicArya eva ahaM waM prawyakRaM na eva praSaMsAmi 
ixaM 

3611
saH pawnyAH kakRe waruNaM hqRtaM puRtaM yuvakaM SayAnaM xqRtvA pawnIM ca xurAcAriNIM mawvA yAvaw eva waM hanwuM pravqwwaH wAvaw eva wasya xqRtO uparyukwaH SlokaH ApawiwaH 
saH pawnyAH kakRe waruNaM hqRtaM puRtaM yuvakaM SayAnaM xqRtvA pawnIM ca xurAcAriNIM mawvA yAvaw eva waM hanwuM pravqwwaH wAvaw eva wasya xqRtO uparyukwaH SlokaH ApawiwaH 
http://sanskrit.inria.fr/cgi-bin/SKT/sktgraph?lex=SH&st=t&us=f&cp=t&text=saH+pawnyAH+kakRe+waruNaM+hqRtaM+puRtaM+yuvakaM+SayAnaM+xqRtvA+pawnIM+ca+xurAcAriNIM+mawvA+yAvaw+eva+waM+hanwuM+pravqwwaH+wAvaw+eva+wasya+xqRtO+uparyukwaH+SlokaH+ApawiwaH+&t=WX&topic=&mode=g&corpmode=&corpdir=&sentno=
3612
viramya saH kiFciw vicArayiwuM pravqwwaH 
viramya saH kiFciw vicArayiwuM pravqwwaH 
http://sanskrit.inria.fr/cgi-bin/SKT/sktgraph?lex=SH&st=t&us=f&cp=t&text=viramya+saH+kiFciw+vicArayiwuM+pravqwwaH+&t=WX&topic=&mode=g&corpmode=&corpdir=&sentno=
3613
wAvaw eva wasya pawnI waM yaWArWawAM praboXiwavawI yaw ayam AvayoH AwmajaH eva aswi 
wAvaw eva wasya pawn

3637
wasmin samaye cOraH gqhe na AsIw 
wasmin samaye cOraH gqhe na AsIw 
http://sanskrit.inria.fr/cgi-bin/SKT/sktgraph?lex=SH&st=t&us=f&cp=t&text=wasmin+samaye+cOraH+gqhe+na+AsIw+&t=WX&topic=&mode=g&corpmode=&corpdir=&sentno=
3638
caNdikA cOramAwaraM kaWiwavawI 
caNdikA cOramAwaraM kaWiwavawI 
http://sanskrit.inria.fr/cgi-bin/SKT/sktgraph?lex=SH&st=t&us=f&cp=t&text=caNdikA+cOramAwaraM+kaWiwavawI+&t=WX&topic=&mode=g&corpmode=&corpdir=&sentno=
3639
wava puwraH cOryAramBawaH pUrvaM mAM prArWiwavAn AsIw yaw yaxi saH mama kqpayA cOrye saPalaH BaviRyawi warhi mahyaM BogaM xAsyawi 
wava puwraH cOryAramBawaH pUrvaM mAM prArWiwavAn AsIw yaw yaxi saH mama kqpayA cOrye saPalaH BaviRyawi warhi mahyaM BogaM xAsyawi 
http://sanskrit.inria.fr/cgi-bin/SKT/sktgraph?lex=SH&st=t&us=f&cp=t&text=wava+puwraH+cOryAramBawaH+pUrvaM+mAM+prArWiwavAn+AsIw+yaw+yaxi+saH+mama+kqpayA+cOrye+saPalaH+BaviRyawi+warhi+mahyaM+BogaM+xAsyawi+&t=WX&topic=&mode=g&corpmode=&corpdir=&sentno=
3640
kinwu saH aXunAvaXi na eva axaxA

3665
rAjA wAn apUjayaw 
rAjA wAn apUjayaw 
http://sanskrit.inria.fr/cgi-bin/SKT/sktgraph?lex=SH&st=t&us=f&cp=t&text=rAjA+wAn+apUjayaw+&t=WX&topic=&mode=g&corpmode=&corpdir=&sentno=
3666
ekAM bqhawIM XanapetikAM ca weRAm AvAse prApayiwuM svakIyam ekaM karmacAriNaM preRiwavAn 
ekAM bqhawIM XanapetikAM ca weRAm AvAse prApayiwuM svakIyam ekaM karmacAriNaM preRiwavAn 
http://sanskrit.inria.fr/cgi-bin/SKT/sktgraph?lex=SH&st=t&us=f&cp=t&text=ekAM+bqhawIM+XanapetikAM+ca+weRAm+AvAse+prApayiwuM+svakIyam+ekaM+karmacAriNaM+preRiwavAn+&t=WX&topic=&mode=g&corpmode=&corpdir=&sentno=
3667
mArge karmacArI wAn sAXUn apqcCaw 
mArge karmacArI wAn sAXUn apqcCaw 
http://sanskrit.inria.fr/cgi-bin/SKT/sktgraph?lex=SH&st=t&us=f&cp=t&text=mArge+karmacArI+wAn+sAXUn+apqcCaw+&t=WX&topic=&mode=g&corpmode=&corpdir=&sentno=
3668
kaWyawAm 
kaWyawAm 
http://sanskrit.inria.fr/cgi-bin/SKT/sktgraph?lex=SH&st=t&us=f&cp=t&text=kaWyawAm+&t=WX&topic=&mode=g&corpmode=&corpdir=&sentno=
3669
mahAwmA BavawaH kaWaM jAnAwi 
mahAwmA

...

3839
 rAmaH prajApawisamaH SrImAn XAwA ripuniRUxanaH jIvalokasya rakRiwA Xarmasya ca parirakRiwA aswi 
 rAmaH prajApawisamaH SrImAn XAwA ripuniRUxanaH jIvalokasya rakRiwA Xarmasya ca parirakRiwA aswi 
http://sanskrit.inria.fr/cgi-bin/SKT/sktgraph?lex=SH&st=t&us=f&cp=t&text=+rAmaH+prajApawisamaH+SrImAn+XAwA+ripuniRUxanaH+jIvalokasya+rakRiwA+Xarmasya+ca+parirakRiwA+aswi+&t=WX&topic=&mode=g&corpmode=&corpdir=&sentno=
3840
 rAmaH svasya Xarmasya svajanasya ca rakRiwA vexavexAfgawawvajFaH Xanurvexe ca niRTiwaH aswi 
 rAmaH svasya Xarmasya svajanasya ca rakRiwA vexavexAfgawawvajFaH Xanurvexe ca niRTiwaH aswi 
http://sanskrit.inria.fr/cgi-bin/SKT/sktgraph?lex=SH&st=t&us=f&cp=t&text=+rAmaH+svasya+Xarmasya+svajanasya+ca+rakRiwA+vexavexAfgawawvajFaH+Xanurvexe+ca+niRTiwaH+aswi+&t=WX&topic=&mode=g&corpmode=&corpdir=&sentno=
3841
 rAmaH sarvaSAswrArWawawvajFaH smqwimAn prawiBAvAn sarvalokapriyaH sAXuH axInAwmA vicakRaNaH ca aswi 
 rAmaH sarvaSAswrArWawawvajFaH smqwimAn prawiBAvAn sarvalokapriyaH sA

3859
AryaBAvapuraskqwaH saH gawvA mahAwmAnaM sawyaparAkramaM BrAwaraM rAmaM ayAcaw
AryaBAvapuraskqwaH saH gawvA mahAwmAnaM sawyaparAkramaM BrAwaraM rAmaM ayAcaw
http://sanskrit.inria.fr/cgi-bin/SKT/sktgraph?lex=SH&st=t&us=f&cp=t&text=AryaBAvapuraskqwaH+saH+gawvA+mahAwmAnaM+sawyaparAkramaM+BrAwaraM+rAmaM+ayAcaw&t=WX&topic=&mode=g&corpmode=&corpdir=&sentno=
3860
wvam eva rAjA XarmajFaH asi iwi vacaH rAmam abravIw
wvam eva rAjA XarmajFaH asi iwi vacaH rAmam abravIw
http://sanskrit.inria.fr/cgi-bin/SKT/sktgraph?lex=SH&st=t&us=f&cp=t&text=wvam+eva+rAjA+XarmajFaH+asi+iwi+vacaH+rAmam+abravIw&t=WX&topic=&mode=g&corpmode=&corpdir=&sentno=
3861
paramoxAraH sumuKaH sumahAyaSAH mahAbalaH rAmaH api piwuH AxeSAw rAjyaM na EcCaw
paramoxAraH sumuKaH sumahAyaSAH mahAbalaH rAmaH api piwuH AxeSAw rAjyaM na EcCaw
http://sanskrit.inria.fr/cgi-bin/SKT/sktgraph?lex=SH&st=t&us=f&cp=t&text=paramoxAraH+sumuKaH+sumahAyaSAH+mahAbalaH+rAmaH+api+piwuH+AxeSAw+rAjyaM+na+EcCaw&t=WX&topic=&mode=g&corpmode=&corpdir=&sen

3881
rAGava XarmanipuNAM XarmacAriNIM SramaNAM SabarIm aBigacCa iwi saH asya kaWayAmAsa
rAGava XarmanipuNAM XarmacAriNIM SramaNAM SabarIm aBigacCa iwi saH asya kaWayAmAsa
http://sanskrit.inria.fr/cgi-bin/SKT/sktgraph?lex=SH&st=t&us=f&cp=t&text=rAGava+XarmanipuNAM+XarmacAriNIM+SramaNAM+SabarIm+aBigacCa+iwi+saH+asya+kaWayAmAsa&t=WX&topic=&mode=g&corpmode=&corpdir=&sentno=
3882
saH mahAwejAH SawrusUxanaH SabarIm aByagacCaw
saH mahAwejAH SawrusUxanaH SabarIm aByagacCaw
http://sanskrit.inria.fr/cgi-bin/SKT/sktgraph?lex=SH&st=t&us=f&cp=t&text=saH+mahAwejAH+SawrusUxanaH+SabarIm+aByagacCaw&t=WX&topic=&mode=g&corpmode=&corpdir=&sentno=
3883
xaSaraWAwmajaH rAmaH SabaryA samyak pUjiwaH
xaSaraWAwmajaH rAmaH SabaryA samyak pUjiwaH
http://sanskrit.inria.fr/cgi-bin/SKT/sktgraph?lex=SH&st=t&us=f&cp=t&text=xaSaraWAwmajaH+rAmaH+SabaryA+samyak+pUjiwaH&t=WX&topic=&mode=g&corpmode=&corpdir=&sentno=
3884
pampAwIre vAnareNa hanumawA safgawaH
pampAwIre vAnareNa hanumawA safgawaH
http://sanskrit.inria.fr/cgi-b

3906
 saHhanumAn paFca senAgragAn sapwa manwrisuwAn ca hawvA SUram akRam api niRpiRya grahaNaM samupAgamaw
 saHhanumAn paFca senAgragAn sapwa manwrisuwAn ca hawvA SUram akRam api niRpiRya grahaNaM samupAgamaw
http://sanskrit.inria.fr/cgi-bin/SKT/sktgraph?lex=SH&st=t&us=f&cp=t&text=+saHhanumAn+paFca+senAgragAn+sapwa+manwrisuwAn+ca+hawvA+SUram+akRam+api+niRpiRya+grahaNaM+samupAgamaw&t=WX&topic=&mode=g&corpmode=&corpdir=&sentno=
3907
 wawaH hanumAn pEwAmahAw varAw aswreNa AwmAnam unmukwaM jFAwvA yanwriNaH wAn rAkRasAn yaxqcCayA marRayan rAvaNaM prApwavAn samupAgamaw 
 wawaH hanumAn pEwAmahAw varAw aswreNa AwmAnam unmukwaM jFAwvA yanwriNaH wAn rAkRasAn yaxqcCayA marRayan rAvaNaM prApwavAn samupAgamaw 
http://sanskrit.inria.fr/cgi-bin/SKT/sktgraph?lex=SH&st=t&us=f&cp=t&text=+wawaH+hanumAn+pEwAmahAw+varAw+aswreNa+AwmAnam+unmukwaM+jFAwvA+yanwriNaH+wAn+rAkRasAn+yaxqcCayA+marRayan+rAvaNaM+prApwavAn+samupAgamaw+&t=WX&topic=&mode=g&corpmode=&corpdir=&sentno=
3908
wawaH mahAkapiH mEWilIM sIwAm qwe

3929
na kiFcix vAwajaM Bayam BaviRyawi 
na kiFcix vAwajaM Bayam BaviRyawi 
http://sanskrit.inria.fr/cgi-bin/SKT/sktgraph?lex=SH&st=t&us=f&cp=t&text=na+kiFcix+vAwajaM+Bayam+BaviRyawi+&t=WX&topic=&mode=g&corpmode=&corpdir=&sentno=
3930
na api jvarakqwaM waWA BaviRyawi 
na api jvarakqwaM waWA BaviRyawi 
http://sanskrit.inria.fr/cgi-bin/SKT/sktgraph?lex=SH&st=t&us=f&cp=t&text=na+api+jvarakqwaM+waWA+BaviRyawi+&t=WX&topic=&mode=g&corpmode=&corpdir=&sentno=
3931
na ca api wawra kRuxBayam BaviRyawi 
na ca api wawra kRuxBayam BaviRyawi 
http://sanskrit.inria.fr/cgi-bin/SKT/sktgraph?lex=SH&st=t&us=f&cp=t&text=na+ca+api+wawra+kRuxBayam+BaviRyawi+&t=WX&topic=&mode=g&corpmode=&corpdir=&sentno=
3932
waWA waskaraBayam api na BaviRyawi 
waWA waskaraBayam api na BaviRyawi 
http://sanskrit.inria.fr/cgi-bin/SKT/sktgraph?lex=SH&st=t&us=f&cp=t&text=waWA+waskaraBayam+api+na+BaviRyawi+&t=WX&topic=&mode=g&corpmode=&corpdir=&sentno=
3933
nagarANi rARtrANi ca XanaXAnyayuwAni BaviRyanwi 
nagarANi rARtrANi ca Xan

In [24]:
## checked manually given files
with open("workdata/lines_submit/analasys/lines_data.txt",encoding = 'utf-8' ,mode ='a') as fw :
    fw.writelines("input errors = 1\n")
    fw.writelines("complete analasys = 3002\n")
    fw.writelines("incomplete analasys = 940\n")

In [30]:
#     s_d = inputline
#     s_c = s_d.replace(" ","+")
#     urlname = ("http://sanskrit.inria.fr/cgi-bin/SKT/sktgraph?lex=SH&st=t&us=f&cp=t&text=" +
#                s_c + "&t=WX&topic=&mode=g&corpmode=&corpdir=&sentno=")

#     print(urlname)
#     time.sleep(2)
#     page = requests.get(urlname)

In [31]:
#     soup = bs(page.text, 'html.parser')
#     soup.prettify()

In [32]:
# table = soup.table
# print(table)

In [33]:
# tablebody = table.find('table',{'class' : 'center'})
# print(tablebody)

In [34]:
# t = pd.DataFrame(columns = ['id','level','color_class','position','chunk_no','word','lemma','pre_verb','morph','colspan','wordlenth','aux_inf'])
# t


In [35]:
# for child in tablebody.children:
#     if(child.name == 'tr') :
#         print(child.name)

In [36]:
# 'id','level','color_class','position','chunk_no','word','lemma','morph','colspan','wordlenth','pre_verb','aux_inf'

In [37]:
# i = 0
# id_= 0
# for child in tablebody.children:
#     if(child.name == 'tr') :
#         if i< 1 : 
#             linechar = []
#             c=0
#             for char in child.children :
#                 linechar.append(char.string)
#                 c += 1
#             i+=1
#             line = "".join(linechar)
#             linechunks = line.split("\xa0")
#             continue
#         position_ = 0
#         j =0
#         for wordtable in child.children:
# #             if(j < 20 ) :
# #                 j += 1
# #                 continue
#             c= 0
#             for ch in linechar[0:position_] :
#                 if(re.match('\xa0',ch)) : 
#                     c+=1
#             if(wordtable.contents):
#                 color_ = wordtable.table.get('class')[0]
#                 colspan_= wordtable.get('colspan')
#                 word_ = wordtable.table.tr.td.string
#                 onclickdatas_ = wordtable.table.tr.td.get('onclick')
#                 for onclickdata_ in onclickdatas_.split("<br>") :
#                     morphslist_ = re.findall(r'{ (.*?) }',onclickdata_)  #.split(' | ')
#                     ldata = str(re.search(r'{.*?}\[(.*)\]',onclickdata_).group(1))
#                     ldata = str(re.sub(r'</?a.*?>|</?i>',"",ldata))

#                     lemmadata = ldata.split(" ")
#                     if len(lemmadata) >1 :
#                         auxi_ = " ".join(lemmadata[1:])
#                     else :
#                         auxi_ = ""
#                     lemmas_ = "".join(lemmadata[0])
#                     lemmalists_ = lemmas_.split("-")
#                     if(len(lemmalists_) > 1) :
#                         preverb_ = ",".join(lemmalists_[0:(len(lemmalists_)-1)])
#                         lemmalist_ = "".join(lemmalists_[-1:]).split("_")
#                     else :
#                         preverb_ = ""
#                         lemmalist_ = "".join(lemmalists_[0]).split("_")
#                     if(len(lemmalist_) > 1):
#                         auxi_ =auxi_ + " sence of lemma = " + "".join(lemmalist_[1:(len(lemmalist_))])
#                         lemma_ = "".join(lemmalist_[0])
#                     else : 
#                         lemma_ = "".join(lemmalist_[0])             
#                     morphs_ = str(morphslist_[0])               
#                     for morph_ in morphs_.split(" | ") : 
#                         t.loc[id_] = [id_,i,color_,position_,c+1,word_,lemma_,preverb_,morph_,colspan_,len(word_),auxi_]
#                         id_ += 1    


#                 position_ += int(colspan_)
#     #                 break
#             else : 
#                 position_ += 1

# #         break
        
#         i = i+1

        
# # print(color_)
# # print(colspan_)
# # print(position_)
# # print(word_)
# # print(onclickdata_)
# # print(ldata)
# # print(morphslist_)
# # print(lemmalists_)
# # print(auxi_)
# # print(lemma_)
# # print(morphs_)
# # print(preverb_)
# # print(linechar) 
# print(line)
# # c= 0
# # for ch in linechar[0:position_] :
# #     if(re.match('\xa0',ch)) : 
# #         c+=1
# # print(c)
# t

In [10]:
# os.makedirs('workdata/lines_submit/wronginput/output/', exist_ok=True)
# os.makedirs('workdata/lines_submit/wrong_lines/output/', exist_ok=True)
# os.makedirs('workdata/lines_submit/correct_lines/output/', exist_ok=True)


# fnames = ["all_uniquelines.txt"]
# count = 0     
# fw1 = open("workdata/lines_submit/wronginput/output/wronginput.txt",encoding = 'utf-8' ,mode ='a')
# for fname in fnames :
#     problem = []
#     problem.append(-1)
#     fr = open("workdata/lines_submit/"+fname,encoding ='utf-8',mode='r')
#     _lines = fr.readlines()
#     fr.close()
    
#     for _line in _lines :
#         if(count+1>25) :
#             break
#         problem = []
#         problem.append(-1)
#         if(_line.startswith("filename : ")) :
#             line1 = _line
#             continue
#         if(_line.startswith("lineno : ")) :
#             line2 = _line
#             continue
#         if(_line.startswith("line : ")) :
#             line3 = _line[7:]
#             count += 1
#             time.sleep(2)
#             _line1 = line3[:-1]
#             _line2 = _line1 # re.sub(r'\d+|\"|\?|\.|\,|\*|\!|_\?|\-|_\ |\'',"",_line1)
#             inputline = _line2
#             if(inputline == " ") :
#                 print("emptyline")
#                 count = count-1
#                 continue
#             s_d = inputline
#             print(count)
#             print(_line1)
#             print(inputline)
# #             fw = codecs.open("workdata/linetabledata/"+ str(count)+".txt",encoding = 'utf-8',mode = 'w')
# #             fw.writelines("given line : " +_line1 + "\n")                                                        #file input
#             s_c = s_d.replace(" ","+")
#             urlname = ("http://sanskrit.inria.fr/cgi-bin/SKT/sktgraph?lex=SH&st=t&us=f&cp=t&text=" +
#                        s_c + "&t=WX&topic=&mode=g&corpmode=&corpdir=&sentno=")

#             print(urlname)
#             page = requests.get(urlname)
#             soup = bs(page.text, 'html.parser')
#         #     soup.prettify()
#             table = soup.table
#             tablebody = table.find('table',{'class' : 'center'})
#             t = pd.DataFrame(columns = ['id','level','color_class','position','chunk_no','word','lemma','pre_verb','morph','colspan','wordlenth','aux_inf'])

#             i = 0
#             id_= 0
#             if not(tablebody) :
#                 fw1.writelines(line1)
#                 fw1.writelines(line2)
#                 fw1.writelines(line3)
#                 fw1.writelines("count =" +str(count)+"\n")
#                 continue
#             for child in tablebody.children:
#                 if(child.name == 'tr') :
#                     if i< 1 : 
#                         linechar = []
#                         c=0
#                         for char in child.children :
#                             linechar.append(char.string)
#                             c += 1
#                         i+=1
#                         line = "".join(linechar)
#                         linechunks = line.split("\xa0")
#                         continue
#                     position_ = 0
#                     j =0
#                     for wordtable in child.children:
#             #             if(j < 20 ) :
#             #                 j += 1
#             #                 continue
#                         c= 0
#                         for ch in linechar[0:position_] :
#                             if(re.match('\xa0',ch)) : 
#                                 c+=1
#                         if(wordtable.contents):
#                             color_ = wordtable.table.get('class')[0]
#                             colspan_= wordtable.get('colspan')
#                             word_ = wordtable.table.tr.td.string
#                             onclickdatas_ = wordtable.table.tr.td.get('onclick')
#                             for onclickdata_ in onclickdatas_.split("<br>") :
#                                 morphslist_ = re.findall(r'{ (.*?) }',onclickdata_)  #.split(' | ')
#                                 ldata = str(re.search(r'{.*?}\[(.*)\]',onclickdata_).group(1))
#                                 ldata = str(re.sub(r'</?a.*?>|</?i>',"",ldata))

#                                 lemmadata = ldata.split(" ")
#                                 if len(lemmadata) >1 :
#                                     auxi_ = " ".join(lemmadata[1:])
#                                 else :
#                                     auxi_ = ""
#                                 lemmas_ = "".join(lemmadata[0])
#                                 lemmalists_ = lemmas_.split("-")
#                                 if(len(lemmalists_) > 1) :
#                                     preverb_ = ",".join(lemmalists_[0:(len(lemmalists_)-1)])
#                                     lemmalist_ = "".join(lemmalists_[-1:]).split("_")
#                                 else :
#                                     preverb_ = ""
#                                     lemmalist_ = "".join(lemmalists_[0]).split("_")
#                                 if(len(lemmalist_) > 1):
#                                     auxi_ =auxi_ + " sence of lemma = " + "".join(lemmalist_[1:(len(lemmalist_))])
#                                     lemma_ = "".join(lemmalist_[0])
#                                 else : 
#                                     lemma_ = "".join(lemmalist_[0])             
#                                 morphs_ = str(morphslist_[0])               
#                                 for morph_ in morphs_.split(" | ") : 
#                                     t.loc[id_] = [id_,i,color_,position_,c+1,word_,lemma_,preverb_,morph_,colspan_,len(word_),auxi_]
#                                     if(re.match(r'grey_back',color_)) :
#                                         problem.append(id_)
#                                     id_ += 1    


#                             position_ += int(colspan_)
#                 #                 break
#                         else : 
#                             position_ += 1

#             #         break

#                     i = i+1
#             if(len(problem)>1) :
#                 fn = "workdata/lines_submit/wrong_lines/output/"+ str(count)+".txt"
#             else :
#                 fn = "workdata/lines_submit/correct_lines/output/"+ str(count)+".txt"
            
#             fw = open(fn,encoding = 'utf-8' ,mode ='w')
#             fw.writelines(line1)
#             fw.writelines(line2)
# #             fw.writelines("given line : " +_line1 + "\n")
#             fw.writelines("input line : " + inputline + "\n")
#             fw.writelines("converted line : "+line + "\n")
#             fw.writelines("converted line char :" + ",".join(linechar) + "\n")
#             if(len(problem)>1) :
#                 _prob = []
#                 for prob in problem[1:]:
#                     _prob.append(str(prob))
#                 fw.writelines("problem_yes ,index = ["+"-".join(_prob) + "]\n")
#             else :
#                 fw.writelines("problem_no" + "\n")
#             fw.close()
#             t.to_csv(fn ,encoding = 'utf-8' , sep=',',index = False,mode = 'a' )
#     #         display(t)
#     #          count += 1 
#     #         if(count > 2) :
#     #             break
#     #     break
# fw1.close()
# # fr = open("data/sansk.txt",'r')
# # print(fr.readlines())
# # fr.close()

1
rAmasya BAryA sIwA wasya anujaH lakRmaNaH ca rAmeNa saha vanam gacCawaH
rAmasya BAryA sIwA wasya anujaH lakRmaNaH ca rAmeNa saha vanam gacCawaH
http://sanskrit.inria.fr/cgi-bin/SKT/sktgraph?lex=SH&st=t&us=f&cp=t&text=rAmasya+BAryA+sIwA+wasya+anujaH+lakRmaNaH+ca+rAmeNa+saha+vanam+gacCawaH&t=WX&topic=&mode=g&corpmode=&corpdir=&sentno=
2
wApasAnAm bAlakAH ca wawra waranwi
wApasAnAm bAlakAH ca wawra waranwi
http://sanskrit.inria.fr/cgi-bin/SKT/sktgraph?lex=SH&st=t&us=f&cp=t&text=wApasAnAm+bAlakAH+ca+wawra+waranwi&t=WX&topic=&mode=g&corpmode=&corpdir=&sentno=
3
vqkReRu lawAH ca sanwi
vqkReRu lawAH ca sanwi
http://sanskrit.inria.fr/cgi-bin/SKT/sktgraph?lex=SH&st=t&us=f&cp=t&text=vqkReRu+lawAH+ca+sanwi&t=WX&topic=&mode=g&corpmode=&corpdir=&sentno=
4
vqkRANAm lawAnAm ca SAKAsu vihagAH kUjanwi
vqkRANAm lawAnAm ca SAKAsu vihagAH kUjanwi
http://sanskrit.inria.fr/cgi-bin/SKT/sktgraph?lex=SH&st=t&us=f&cp=t&text=vqkRANAm+lawAnAm+ca+SAKAsu+vihagAH+kUjanwi&t=WX&topic=&mode=g&corpmode=&corpdir=&sentn